# Проект [u__2__1 b] **Разработка кредитных рейтингов CTF** (Shepitko I)

---

[Содержание](#content)
1. [Импорт библиотек](#libraries)
1. [Загрузка данных](#data_upload)
1. [Features Engineering](#f_engin)
    1. [oil](#oil)
2. [Моделирование](#modeling)
    1. [Корреляции](#corr)
    2. [Регрессии](#regs)
3. [Результаты](#results)



<br>

## Импорт библиотек <a name="libraries"></a>

---


In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import scipy.stats as st

<br>

## Определение функций <a name="functions"></a>

---

In [4]:
def Somers_D(xx,yy):
    if yy.isnull().sum()>0:
        return print('\\nNANS!')

    else :
        x = st.rankdata(xx)
        y = st.rankdata(yy)

        n2const = y.size*(y.size-1)/2
        n=np.zeros(np.unique(y).size)

        for i in range (0,np.unique(y).size):
            for j in range(0,y.size):
                if (y[j] == np.unique(y)[i]):
                    n[i]+=1            
        y_adj = 0

        for ii in range (0,np.unique(y).size):
            y_adj += 0.5*n[ii]*(n[ii]-1)
         
        K = 0

        for kk in range(0,y.size-1):
            for kkk in range(kk+1,y.size):
                K += np.sign(x[kk] - x[kkk])*np.sign(y[kk] - y[kkk])
        return K/(n2const-y_adj)

<br>

## Загрузка данных <a name="data_upload"></a>

---

In [3]:
l__p1 = "/content/drive/MyDrive/Science/u___Universities/u__2__1_RUS__MSU___CMF_2021_2022/u__2__1_b__#Разработка_кредитных_рейтингов_CTF/Data/CTF_expert_ordering_v0.3.xlsx"
l__oil1 = "/content/drive/MyDrive/Science/u___Universities/u__2__1_RUS__MSU___CMF_2021_2022/u__2__1_b__#Разработка_кредитных_рейтингов_CTF/Data/oil___Brent Oil Futures Historical Data.csv"
l__oil2 = "/content/drive/MyDrive/Science/u___Universities/u__2__1_RUS__MSU___CMF_2021_2022/u__2__1_b__#Разработка_кредитных_рейтингов_CTF/Data/oil___Crude Oil WTI Futures Historical Data.csv"


#  --
#
l__ = [ 'main_industry','company_name'
     ,'report_date','observation_date'
     ,'экспертный рейтинг (бизнес)','экспертный рейтинг (андеррайтинг)']
Eord = pd.read_excel(l__p1)
Eord = Eord[l__]
Eord['target'] = 0.5*(Eord['экспертный рейтинг (бизнес)'] + Eord['экспертный рейтинг (андеррайтинг)'])
Eord.loc[Eord['экспертный рейтинг (бизнес)'].isnull(),['target']]

l__dates = np.unique(Eord['report_date'].to_list())

#  -- MACRO : OIL
#
l___ = ['Date','Price']
OIL  = pd.read_csv(l__oil1)
OIL['Date'] = OIL['Date'].map(lambda x : pd.to_datetime(x, format="%b %d, %Y"))
OIL = OIL[l___]



print( '\n Eord : ',Eord.shape
      ,'\n',Eord.dtypes
      ,'\n OIL : ',OIL.shape
      ,'\n',OIL.dtypes
      )



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Science/u___Universities/u__2__1_RUS__MSU___CMF_2021_2022/u__2__1_b__#Разработка_кредитных_рейтингов_CTF/Data/CTF_expert_ordering_v0.3.xlsx'

<br>

## Features Engineering <a name="f_engin"></a>

---

<br>

### oil <a name="oil"></a>

---

In [24]:
l_vars = []

tmp = OIL.copy()

for _dt in l__dates:
    # MCR1_03
    _dt_x = _dt + relativedelta(months=-12)
    t___ = tmp.loc[(tmp['Date']<_dt) & (tmp['Date']>=_dt_x),'Price']
    vara1 = round(np.std(t___)/np.mean(t___),4)
    # MCR2_03
    _dt_x = _dt + relativedelta(months=12)
    t___ = tmp.loc[(tmp['Date']<_dt_x) & (tmp['Date']>=_dt),['Price']].reset_index(drop=True)
    vara2 = round((t___.loc[(t___.shape[0]-1),'Price']/t___.loc[0,'Price']) - 1,4)

    print(' dt : ',_dt,' MCR1_03 : ',vara1,' MCR3_03 : ',vara2)
    l_vars.append(['oil',_dt,vara1,vara2])

#
#
tmp = ALUM.copy()

for _dt in l__dates:
    # MCR1_03
    _dt_x = _dt + relativedelta(months=-12)
    t___ = tmp.loc[(tmp['Date']<_dt) & (tmp['Date']>=_dt_x),'Price']
    vara1 = round(np.std(t___)/np.mean(t___),4)
    # MCR2_03
    _dt_x = _dt + relativedelta(months=12)
    t___ = tmp.loc[(tmp['Date']<_dt_x) & (tmp['Date']>=_dt),['Price']].reset_index(drop=True)
    vara2 = round((t___.loc[(t___.shape[0]-1),'Price']/t___.loc[0,'Price']) - 1,4)

    print(' dt : ',_dt,' MCR1_03 : ',vara1,' MCR3_03 : ',vara2)
    l_vars.append(['aluminium',_dt,vara1,vara2])














df_vars = pd.DataFrame(l_vars,columns=['main_industry','report_date','MCR1_03','MCR3_03'])
df_vars

 dt :  2008-01-01 00:00:00  MCR1_03 :  0.1578  MCR3_03 :  1.1461
 dt :  2009-01-01 00:00:00  MCR1_03 :  0.2887  MCR3_03 :  -0.398
 dt :  2010-01-01 00:00:00  MCR1_03 :  0.1937  MCR3_03 :  -0.1544
 dt :  2011-01-01 00:00:00  MCR1_03 :  0.0725  MCR3_03 :  -0.1168
 dt :  2012-01-01 00:00:00  MCR1_03 :  0.0627  MCR3_03 :  0.0092
 dt :  2013-01-01 00:00:00  MCR1_03 :  0.0694  MCR3_03 :  0.0151
 dt :  2014-01-01 00:00:00  MCR1_03 :  0.0391  MCR3_03 :  0.9348
 dt :  2015-01-01 00:00:00  MCR1_03 :  0.1461  MCR3_03 :  0.5134
 dt :  2016-01-01 00:00:00  MCR1_03 :  0.15  MCR3_03 :  -0.3449
 dt :  2017-01-01 00:00:00  MCR1_03 :  0.1495  MCR3_03 :  -0.1503
 dt :  2018-01-01 00:00:00  MCR1_03 :  0.0906  MCR3_03 :  0.2374
 dt :  2019-01-01 00:00:00  MCR1_03 :  0.0959  MCR3_03 :  -0.168
 dt :  2020-01-01 00:00:00  MCR1_03 :  0.0645  MCR3_03 :  0.279


,main_industry,report_date,MCR1_03,MCR3_03
0,oil,2008-01-01,0.1578,1.1461
1,oil,2009-01-01,0.2887,-0.3980
2,oil,2010-01-01,0.1937,-0.1544
3,oil,2011-01-01,0.0725,-0.1168
4,oil,2012-01-01,0.0627,0.0092
5,oil,2013-01-01,0.0694,0.0151
6,oil,2014-01-01,0.0391,0.9348
7,oil,2015-01-01,0.1461,0.5134
8,oil,2016-01-01,0.1500,-0.3449
9,oil,2017-01-01,0.1495,-0.1503


<br>

## Моделирование <a name="modeling"></a>

---

In [27]:
DEV_ = pd.merge(Eord[['main_industry','report_date','company_name','target']]
               , df_vars
               , how='left'
               , on = ['main_industry','report_date']
               )

DEV = DEV_.loc[DEV_['target'].notnull(),:].reset_index(drop=True)
print('\n',DEV_.shape[0]
    ,'\n',DEV.shape[0]
      )


 183 
 169


<br>

### Корреляции <a name="corr"></a>

---

In [32]:
l_vars = ['MCR1_03','MCR3_03']

for vara in l_vars : 
    tmp = DEV.loc[DEV[vara].notnull(),:]
    c_ = round(np.corrcoef(tmp[vara],tmp['target'])[0,1],4)
    sd_ = Somers_D(tmp[vara],tmp['target'])
    print('\nФактор : ',vara, ' c : ',c_,' somers d : ',sd_)



Фактор :  MCR1_03  c :  0.2208  sd :  0.18998978549540346

Фактор :  MCR3_03  c :  -0.1461  sd :  -0.15526046986721145


In [26]:
DEV.head(50)

,main_industry,report_date,company_name,target,MCR1_03,MCR3_03
0,NaN,2010-01-01,Gunvor Group,4.0,NaN,NaN
1,NaN,2011-01-01,Gunvor Group,4.0,NaN,NaN
2,NaN,2012-01-01,Gunvor Group,4.0,NaN,NaN
3,NaN,2013-01-01,Gunvor Group,4.0,NaN,NaN
4,NaN,2014-01-01,Gunvor Group,4.0,NaN,NaN
5,NaN,2015-01-01,Gunvor Group,4.0,NaN,NaN
6,NaN,2018-01-01,Gunvor Group,4.0,NaN,NaN
7,NaN,2019-01-01,Gunvor Group,4.0,NaN,NaN
8,metals,2008-01-01,MMK Trading,NaN,NaN,NaN
9,metals,2009-01-01,MMK Trading,NaN,NaN,NaN


<br>

### Регрессии <a name="regs"></a>

---

In [37]:
arr1 = pd.Series([1,2,124])
arr2 = pd.Series([-1,-2,-3])

In [38]:
Somers_D(arr1,arr2)

-1.0